In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly_express as px
import os 
import numpy as np
# csv to dataframe
df = pd.read_csv('combined.csv')

df = df.drop(['OffenseCode', 'LocationStreetNumber', 'LocationDirectional', 'CaseStatus','OCANumber'], axis=1)


# if the offense description is empty, fill it with the string " no offense description"





In [14]:

df.head(5)


,IncidentFromDate,IncidentFromTime,IncidentToDate,IncidentToTime,Offense Description,CaseDisposition,LocationCode,PatrolZone,LocationLandmark,LocationStreet,LocationLatitude,LocationLongitude,CreatedSource,Unnamed: 18
0,1/1/2010,1:55:00,1/1/2010,2:58:00,Miscellaneous Offenses,NaN,ONCAMRES,Z1,Center Street Apartments North 132 Z1,"Hemphill Avenue, NW",NaN,NaN,1.0,NaN
1,1/1/2010,1:36:00,NaN,NaN,Miscellaneous Offenses,NaN,NONCAM,Z4,NaN,NaN,NaN,NaN,1.0,NaN
2,1/1/2010,2:00:00,1/1/2010,4:00:00,Larceny – From Building,NaN,NONCAM,Z2,Sigma Nu Fraternity 325 Z2,"Fowler Street, NW",NaN,NaN,1.0,NaN
3,1/2/2010,6:15:00,NaN,NaN,Miscellaneous Offenses,NaN,ONCAM,Z2,North Avenue Apartments,"North Avenue, NW",NaN,NaN,1.0,NaN
4,1/3/2010,1:29:00,NaN,NaN,Miscellaneous Offenses,Prosecution Declined,PUB,Z4,"Marietta Street, NW b/w Northside Drive, NW & ...",NaN,NaN,NaN,1.0,NaN


In [15]:
# # if there's no locationlatitude and longitude, fill it with NaN
df['LocationLatitude'] = df['LocationLatitude'].fillna(0)
df['LocationLongitude'] = df['LocationLongitude'].fillna(0)

# combine incident from date with incident from time to get the date and time
df['DateTime'] = df['IncidentFromDate'] + ' ' + df['IncidentFromTime']


# move incident fromdate to the datetime value 
df['IncidentFromDate'] = df['DateTime']
df.drop(['IncidentFromTime'], axis=1, inplace=True)
df.drop(['DateTime'], axis=1, inplace=True)

In [16]:
df.head(5)

,IncidentFromDate,IncidentToDate,IncidentToTime,Offense Description,CaseDisposition,LocationCode,PatrolZone,LocationLandmark,LocationStreet,LocationLatitude,LocationLongitude,CreatedSource,Unnamed: 18
0,1/1/2010 1:55:00,1/1/2010,2:58:00,Miscellaneous Offenses,NaN,ONCAMRES,Z1,Center Street Apartments North 132 Z1,"Hemphill Avenue, NW",0,0,1.0,NaN
1,1/1/2010 1:36:00,NaN,NaN,Miscellaneous Offenses,NaN,NONCAM,Z4,NaN,NaN,0,0,1.0,NaN
2,1/1/2010 2:00:00,1/1/2010,4:00:00,Larceny – From Building,NaN,NONCAM,Z2,Sigma Nu Fraternity 325 Z2,"Fowler Street, NW",0,0,1.0,NaN
3,1/2/2010 6:15:00,NaN,NaN,Miscellaneous Offenses,NaN,ONCAM,Z2,North Avenue Apartments,"North Avenue, NW",0,0,1.0,NaN
4,1/3/2010 1:29:00,NaN,NaN,Miscellaneous Offenses,Prosecution Declined,PUB,Z4,"Marietta Street, NW b/w Northside Drive, NW & ...",NaN,0,0,1.0,NaN


In [17]:
#convert incidentfromdate to datetime

df['IncidentFromDate'] = pd.to_datetime(df['IncidentFromDate'])

#replace Offense Description value 1 with no offense description

df['Offense Description'] = df['Offense Description'].replace(1, 'no offense description')




#using plotly, plot the bar graph of the number of crimes per year, and count 

fig = px.histogram(df, x=df['IncidentFromDate'].dt.year, text_auto=True, color_discrete_sequence=['#1f77b4'])

fig.update_layout(bargap=0.2 )

fig.write_image('CrimePerYear.png')

In [18]:
# filter by crime type received from 'Offense Description

df['Offense Description'] = df['Offense Description'].str.lower()

# change nan with no offense description
df.loc[df['Offense Description'].isna(), 'Offense Description'] = 'no offense description'

# replace specials with - 
df['Offense Description'] = df['Offense Description'].str.replace('�', '-')


Crime_Type_List = sorted(list(set(df['Offense Description'].values.tolist())))

print(Crime_Type_List)

['1', 'aggravated assault', 'aggravated assault/battery - gun', 'aggravated assault/battery - other weapon', 'aggravated assault/battery – gun', 'aggravated assault/battery – other weapon', 'all other larceny', 'all other offenses', 'amphetamine - possession of ', 'amphetamine - sales of', 'amphetamine – possession of ', 'arson', 'arson offense free text', 'assault/battery offense free text', 'bad checks', 'bomb threat', 'burglary  - forced entry - nonresidence', 'burglary  - forced entry - residence', 'burglary  – forced entry – nonresidence', 'burglary  – forced entry – residence', 'burglary - no forced entry - residence', 'burglary - no forced entry -nonresidence', 'burglary – no forced entry – residence', 'burglary – no forced entry –nonresidence', 'burglary/breaking & entering', 'carrying a concealed weapon', 'carrying a prohibited weapon', 'child neglect', 'cocaine - possession of', 'cocaine free text', 'cocaine – possession of', 'contributing to the delinquency of a minor', 'cou

In [19]:
# map year 2021 based on crimetype

# for this data to be valuable, get rid of the crimetype 'non-crime', no offense description 


df1 = df[df['Offense Description'] != 'non-crime (not reported to state)']
df1= df1[df1['Offense Description'] != 'no offense description']
df1= df1[df1['Offense Description'] != 'non-crime']
df1= df1[df1['Offense Description'] != 'all other offenses']
df1 = df1[df1['Offense Description'] != 'miscellaneous offenses']

fig2 = px.histogram(df1, x=df1['Offense Description'], color_discrete_sequence=['#000000'], width = 800, height=800)

fig2.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
    paper_bgcolor="LightSteelBlue",
)

fig2.write_image('OffenseType.png')